### Deploy and Interact with Consent based Smart Contract
The aim of this notebook is to compile, deploy and interact with consent based smart contract

# Compile contract from file

In [36]:
# Import libraries
import json
import web3
from datetime import datetime
from web3 import Web3

# import py_solc_x as px
import solcx
from web3.contract import Contract
import os

# from brownie import  web3

# import icd10

In [37]:
# Read in LUCE contract code
# solcx.install_solc('0.8.0')
print(os.getcwd())
consent_fp = r"data\UnifiedConsentModel.sol"
# consent_fp_relative = r"jupyter\\data\\UnifiedConsentModel.sol"
with open(consent_fp) as file:
    contract_source_code = file.read()

# Compile & Store Compiled source code
# print(contract_source_code)
solcx_version = "0.8.17"
# solcx_version = '0.5.16'
# solcx.install_solc(solcx_version)
# compiled_sol = solcx.compile_source(source=contract_source_code,
#   solc_binary="/snap/bin/solc")
compiled_sol = solcx.compile_source(
    source=contract_source_code,
    # solc_binary='/usr/bin/solc',
    # optimize_runs=200,
    output_values=["abi", "bin", "bin-runtime"],
    optimize=True,
    solc_version=solcx_version,
)

# compiled_sol = solcx.compile_files(source_files=consent_fp,
#                                     # solc_binary='/usr/bin/solc',
#                                     # optimize_runs=200,
#                                     output_values=["abi", "bin",'bin-runtime'],
#                                     # optimize=True,
#                                     solc_version = solcx_version
#                                     )

print(compiled_sol.keys())
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol["<stdin>:ConsentCode"]
# contract_interface = compiled_sol['data/ConsentContractDate.sol:ConsentCode']
print(contract_interface.keys())
# contract_interface
# Extract abi and bytecode
abi = contract_interface["abi"]
bytecode = contract_interface["bin"]
### Deploy
startTime = datetime.now()
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

# compiled_contract = web3.eth.compile_contract(contract_source)
# abi = compiled_contract["abi"]
# bytecode = compiled_contract["bytecode"]
# contract_address = web3.eth.contract(abi=abi, bytecode=bytecode).deploy()

# assert True is w3.isConnected()
# Use local Ganache GUI on macOS
# w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]
w3.eth.accounts
# The default `eth.defaultAccount` address is used as the default "from" property for request_1_address dictionaries if no other explicit "from" property is specified.
# Create contract blueprint
contract = w3.eth.contract(abi=abi, bytecode=bytecode)
# Submit the request_1_address that deploys the contract
provider_address = w3.eth.accounts[0]
provider_address_sum = Web3.to_checksum_address(provider_address)

request_1_address = w3.eth.accounts[1]
request_1_address_sum = Web3.to_checksum_address(request_1_address)

request_2_address = w3.eth.accounts[2]
request_2_address_sum = Web3.to_checksum_address(request_1_address)

tx_hash = contract.constructor().transact({"from": provider_address, "gas": int(1e7)})


tx_hash
### Obtain Transcation Receipt
# Wait for the request_1_address to be mined, and get the request_1_address receipt
# tx_receipt = w3.eth.waitForrequest_1_addressReceipt(tx_hash)

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=120, poll_latency=0.1)

# We obtain the block number under which it is deployed
# global contract_block
contract_block = w3.eth.block_number
print("The contract is deployed with block number", contract_block, ".")
# With obtain the final address of the contract

# global contract_address
contract_address = tx_receipt.contractAddress

print("The contract has the address", contract_address)
gas_limit = int(1e7)
provider_dict = {
    "from": provider_address,
    "to": contract_address,
    "value": 0,
    "gas": gas_limit,
    # 'gasPrice': w3.eth.gas_price*0.1,
    # 'nonce': 33 ,
}

requester_1_dict = {
    "from": request_1_address,
    "to": contract_address,
    "value": 0,
    "gas": gas_limit,
    # 'gasPrice': w3.eth.gas_price*0.1,
    # 'nonce': 33 ,
}

requester_2_dict = {
    "from": request_2_address,
    "to": contract_address,
    "value": 0,
    "gas": gas_limit,
    # 'gasPrice': w3.eth.gas_price*0.1,
    # 'nonce': 33 ,
}
### Interact with contract
# Create python instance of deployed contract
# caddress = '0x3FEAfC9084e95BC5B07FBbBd197Af22422A46019'
contract = w3.eth.contract(
    address=contract_address,
    abi=abi,
    # bytecode=bytecode,
)
# Extract default accounts created by ganache
accounts = w3.eth.accounts
accounts

d:\WorkSpace\Maastricht\LUCE\DecentralizedHealthcareBackend\jupyter
dict_keys(['<stdin>:ConsentCode'])
dict_keys(['abi', 'bin', 'bin-runtime'])
The contract is deployed with block number 2982 .
The contract has the address 0x467a9cfa32D10BF8AeBc74a208795F2B4f34e8Db


['0x84dF8A457982333C0893b968872F57b963243307',
 '0x159f33a7b5a9fB939b8AC6f8CE2E54e7eaE13bE0',
 '0x68C0eC5c56f0EDe43affA748e5D4aFd87f31513d']

# Test cases

## Disease Name

In [38]:
import re

pattern = r"^[A-Z][0-9]{2}$"
pattern_compiled = re.compile(pattern)


def diseaseCode2Int(code: str) -> int:
    # r = fx.UInt16(0)
    # print(bin(r))
    # since A00.0 is a correct code, so plus 1 for each level
    if not pattern_compiled.match(code):
        print(f"The string {code} not matches the pattern")
        return 0
    chapter_str = code[0]
    group_str = code[1:3]
    category_str = code[-1]
    result = 0
    if chapter_str == "*":
        return 0
        # chapter_int = 0
    else:
        chapter_int = int(ord(code[0]) - ord("A")) + 1
        b_chapter = (chapter_int) << 11

    if group_str == "**" or group_str == "*":
        return b_chapter
    else:
        group_int = int(code[1:3]) + 1
        b_group = (group_int) << 4

    # if category_str == "*":
    #     return b_chapter + b_group
    # else:
    #     category_int = int(code[-1]) + 1
    # return b_chapter + b_group + category_int
    return b_chapter + b_group

def int2DiseaseCode(code: int) -> str:
    if code == 0:
        return "*00.0"
    chapter_int = code >> 11
    chapter_str = chr(chapter_int + ord("A") - 1)
    group_int = (code >> 4) & 0b1111111
    group_str = str(group_int - 1).zfill(2)
    category_int = code & 0b1111
    category_str = str(category_int - 1)
    return f"{chapter_str}{group_str}.{category_str}"

aCode = "A00.1"
print(diseaseCode2Int(aCode))
print(bin(diseaseCode2Int(aCode)))

aCode = "*00.1"
print(bin(diseaseCode2Int(aCode)))

aCode = "A**.1"
print(bin(diseaseCode2Int(aCode)))
aCode = "A00.*"
print(bin(diseaseCode2Int(aCode)))

aCode = "000.*"
print(bin(diseaseCode2Int(aCode)))

The string A00.1 not matches the pattern
0
The string A00.1 not matches the pattern
0b0
The string *00.1 not matches the pattern
0b0
The string A**.1 not matches the pattern
0b0
The string A00.* not matches the pattern
0b0
The string 000.* not matches the pattern
0b0


In [39]:
# print(contract.address)
# print(contract.functions)


# print(contract.checkAccess(requester_1_dict))


class BooleanItems:
    ClinicalProfessionals = False
    AcademicProfessionals = False
    ReferenceOrControlMaterial = False
    MethodsDevelopment = False
    PopulationsResearch = False
    AncestryResearch = False
    FundamentalBioResearch = False
    DrugDevelopmentResearch = False
    AgeCategoriesResearch = False
    GenderCategoriesResearch = False
    ProfitPurpose = False
    ProfitMakingProfessionals = False
    FormalApprovalRequired = False

    def to_list(self):
        return [
            self.ClinicalProfessionals,
            self.AcademicProfessionals,
            self.ReferenceOrControlMaterial,
            self.MethodsDevelopment,
            self.PopulationsResearch,
            self.AncestryResearch,
            self.FundamentalBioResearch,
            self.DrugDevelopmentResearch,
            self.AgeCategoriesResearch,
            self.GenderCategoriesResearch,
            self.ProfitPurpose,
            self.ProfitMakingProfessionals,
            self.FormalApprovalRequired,
        ]

    def to_int(self):
        result = 0
        bool_list = self.to_list()
        for i in range(len(bool_list)):
            result += bool_list[i] << i
        return result

    def decode_from_int(self, int_value):
        bool_list = []
        for i in range(13):
            bool_list.append((int_value >> i) & 1)
        self.ClinicalProfessionals = bool_list[0]
        self.AcademicProfessionals = bool_list[1]

        self.ReferenceOrControlMaterial = bool_list[2]
        self.MethodsDevelopment = bool_list[3]
        self.PopulationsResearch = bool_list[4]
        self.AncestryResearch = bool_list[5]
        self.FundamentalBioResearch = bool_list[6]
        self.DrugDevelopmentResearch = bool_list[7]
        self.AgeCategoriesResearch = bool_list[8]
        self.GenderCategoriesResearch = bool_list[9]
        self.ProfitPurpose = bool_list[10]
        self.ProfitMakingProfessionals = bool_list[11]
        self.FormalApprovalRequired = bool_list[12]
        return self

    def show_true_items(self):
        result = []
        for k, v in self.__dict__.items():
            if v:
                result.append(k)
        return result


role_provider = 1
role_requester = 2


class Person:
    address = 0
    name = ""
    description = ""

    boolItems = BooleanItems()

    disease_items = []
    country_names = []
    group_names = 0
    start_year = 0
    start_month = 0
    start_day = 0
    months = 0

    contract = None
    person_dict = None
    role = 0

    estimate_gas = False

    country_name_code_dict = json.load(open("data/countries_enrich.json", "r"))
    group_index_dict = json.load(open("data/group_index.json", "r"))
    country_code_name_dict = {v["index"]: k for k, v in country_name_code_dict.items()}

    def __init__(self, name, description, contract, person_dict):
        self.name = name
        self.address = person_dict["from"]
        self.description = description
        self.contract = contract
        self.person_dict = person_dict

    def upload_simple_items(self):
        upload_func = self.contract.functions.UploadSimpleItems(
            self.role, self.address, self.boolItems.to_int()
        )
        return self.forward(upload_func)

    def display_simple_items(self):
        func = self.contract.functions.DisplaySimpleItems(self.role, self.address)
        if self.estimate_gas:
            gas = func.estimate_gas()
            print(f"Estimated gas cost for displaySimpleItems: {gas}")
        result = func.call()
        boolItems = BooleanItems()
        boolItems.decode_from_int(result)
        # print(f"displaySimpleItems is {boolItems.show_true_items()}")

    def upload_area_code(self):
        country_codes = [
            self.country_name_code_dict[c]["index"] for c in self.country_names
        ]
        # print("UploadCountryItems", country_codes)
        group_codes = [self.group_index_dict[g] for g in self.group_names]
        country_group_codes = []
        for c in self.country_names:
            if c not in self.country_name_code_dict:
                print(f"{c} not in country_dict")
                return
            groups = self.country_name_code_dict[c]["groups"]
            if len(groups) == 0:
                country_group_codes.append(0)
            else:
                country_group_code = 0
                for g in groups:
                    country_group_code += self.group_index_dict[g]
                country_group_codes.append(country_group_code)
        group_code = sum(group_codes)
        assert len(country_codes) == len(country_group_codes)
        func = self.contract.functions.UploadAreaCode(
            self.role,
            self.address,
            group_code,
            country_codes,
            country_group_codes,
        )
        # print("UploadAreaCode role", self.role)
        return self.forward(func)

    def display_area_codes(self):
        (
            Group_Code,
            Country_Codes,
            Country_Group_Codes,
        ) = self.contract.functions.DisplayAreaCode(self.role, self.address).call()
        # groupp_code is a int of the sum of the value of group_index_dict, decode it according to the group_index_dict
        group_names = []
        for k, v in self.group_index_dict.items():
            if v & Group_Code:
                group_names.append(k)
        # print("displayAreaCodes", Country_Codes)
        # print("Country_Group_Codes", Country_Group_Codes)
        country_names = [self.country_code_name_dict[c] for c in Country_Codes]
        result = {"group_names": group_names, "country_codes": country_names}
        print(f"displayAreaCodes is {result}")

    def upload_disease_items(self):
        disease_codes = [diseaseCode2Int(d) for d in self.disease_items]
        func = self.contract.functions.UploadDiseaseCode(
            self.role, self.address, disease_codes
        )
        return self.forward(func)

    def display_disease_items(self):
        disease_codes = self.contract.functions.DisplayDiseaseCode(
            self.role, self.address
        ).call()
        disease_names = []
        for d in disease_codes:
            disease_names.append(int2DiseaseCode(d))
        result = {"disease_names": disease_names}
        print(f"displayDiseaseItems is {result}")

    def upload_date(self):
        func = self.contract.functions.UploadDate(
            self.role,
            self.address,
            self.Start_Year,
            self.Start_Month,
            self.Start_Day,
            self.Months,
        )
        return self.forward(func)

    def display_date(self):
        (
            Start_Year,
            Start_Month,
            Start_Day,
            Months,
        ) = self.contract.functions.DisplayDate(self.role, self.address).call()
        print(
            f"displayDate: Start_Year is {Start_Year}, Start_Month is {Start_Month}, Start_Day is {Start_Day}, Months {Months}"
        )
        return Start_Year, Start_Month, Start_Day, Months

    def forward(self, func, call=False):
        if self.estimate_gas:
            gas = func.estimate_gas()
            return gas
        else:
            if call:
                return func.call()
            else:
                func.transact(self.person_dict)

    def __str__(self):
        return f"Person: {self.name},  {self.description}, {self.role}"


class Provider(Person):
    # def __init__(self):
    #     # super().__init__(*args,**kwargs)
    #     self.role = 1
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.role = role_provider


class Requester(Person):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.role = role_requester

    # def __init__(self):
    #     self.role = 2

    def request_access(self, provider: Provider):
        if provider.role != 1:
            print("requestAccess: provider is not a Provider")
            return
        func = self.contract.functions.AccessData(provider.address, self.address)
        return self.forward(func, True)



In [40]:

provider1 = Provider(
    name="Provider1",
    description="Provider1",
    contract=contract,
    person_dict=provider_dict,
)
# print(provider1)
provider1.boolItems.ClinicalProfessionals = True
provider1.boolItems.AncestryResearch = True
provider1.estimate_gas = True

simple_gase = provider1.upload_simple_items()
# provider1.display_simple_items()

provider1.country_names = ["AFG", "CHN", "USA"]
provider1.group_names = ["G77", "G20"]
area_gas = provider1.upload_area_code()
# provider1.display_area_codes()

provider1.disease_items = ["A10", "C00"]
disease_gas = provider1.upload_disease_items()
# provider1.display_disease_items()

provider1.Start_Year = 2021
provider1.Start_Month = 1
provider1.Start_Day = 1
provider1.Months = 12
date_gas = provider1.upload_date()
# provider1.display_date()

requester1 = Requester(
    name="Requester1",
    description="Requester1",
    contract=contract,
    person_dict=requester_1_dict,
)
requester1.estimate_gas = True
requester1.boolItems.ClinicalProfessionals = True
requester1.boolItems.AncestryResearch = True
requester1.boolItems.ProfitPurpose = True
simple_gas_r = requester1.upload_simple_items()
requester1.country_names = ["AFG", "CHN", "USA"]
# requester1.display_simple_items()
access_gas = requester1.request_access(provider1)

print(f"simple_gas {simple_gase}, area_gas {area_gas}, disease_gas {disease_gas}, date_gas {date_gas}, simple_gas_r {simple_gas_r}, access_gas {access_gas}")


simple_gas 46655, area_gas 219684, disease_gas 122960, date_gas 47489, simple_gas_r 46819, access_gas 36050


In [45]:
# # from web3 import Web3

# # # Connect to your Ethereum node
# # web3 = Web3(Web3.HTTPProvider("your_ethereum_node_url"))

# # # Replace with your contract address and ABI
# # contract_address = "0xYourContractAddress"
# # contract_abi = [...]  # Replace with your contract's ABI

# # Replace with the function you want to estimate gas for
# function_name = "DisplayAreaCode"

# # Create a contract instance
# # contract = web3.eth.contract(address=contract_address, abi=contract_abi)

# # Build the transaction
# transaction = contract.functions[function_name](
#     role_provider, provider_address
# ).build_transaction(
#     {
#         "from": provider_address,
#         "gas": 200000,  # You can set an arbitrary initial gas limit
#     }
# )

# # Estimate gas cost
# # gas_estimate = transaction.estimate_gas()
# gas_estimate = contract.estimate_gas(transaction)

# print(f"Gas Estimate for {function_name}: {gas_estimate}")

# # Send a transaction and get the transaction hash
# transaction_hash = web3.eth.send_transaction(transaction)

# # Wait for the transaction to be mined
# transaction_receipt = web3.eth.waitForTransactionReceipt(transaction_hash)

# # Get the gas used (execution cost)
# gas_used = transaction_receipt["gasUsed"]

# print(f"Gas Used for {function_name}: {gas_used}")

AttributeError: 'Contract' object has no attribute 'estimate_gas'

In [ ]:
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, False, False, False, False, False, False, False).transact(provider_dict)
tx_hash = contract.functions.UploadProviderDate(provider_address, 2023, 6, 27, 6).transact(provider_dict)
tx_hash = contract.functions.giveResearchPurpose(request_1_address, True, True, False, False, False).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, True, False, True, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, False, True).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterDate(request_1_address, 2023,6,29,2023,8,17).transact(requester_1_dict)

## ***positive test***
provider_disease_code_list = ["A00.1", "C00.*","B**.*","*92.0"]
requester_disease_code_list = ["A00.1", "C00.1","B00.3", "A00.4"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]

result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.AccessData(provider_address, request_1_address).call()
result = contract.functions.AccessData(provider_address, request_1_address).call()
print(result)
assert result == 0



ABIFunctionNotFound: ("The function 'displayProviderDiseasePurpose' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')

In [ ]:
provider_disease_code_list = ["A00.1", "C00.*","B**.*"]
requester_disease_code_list = ["B00.3"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]
tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_2_address,requester_disease_code_list).transact(requester_2_dict)

result = contract.functions.AccessDisease(provider_address, request_2_address).call()
print(result)
assert result == 0

# ***negitive test***
# _OpenToGeneralResearchAndClinicalCare is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, True, True, True).transact(provider_dict)
provider_disease_code_list = ["A00.1", "C00.*","B**.*"]
requester_disease_code_list = ["D00.3", "A00.4"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result != 0
# _OpenToHMBResearch is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, False, True, True).transact(provider_dict)
provider_disease_code_list = ["A00.1", "C00.*","B**.*"]
requester_disease_code_list = ["C00.3", "A00.4"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result != 0

# _OpenToDiseaseSpecific is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, False).transact(provider_dict)
tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)
result = contract.functions.displayProviderPrimaryCategorye(provider_address).call()
print(result)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result != 0

# code not equal is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)
result = contract.functions.displayProviderPrimaryCategorye(provider_address).call()
print(result)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result == 0

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, False, False, False, False, False, False, False).transact(provider_dict)
# tx_hash = contract.functions.UploadProviderDate(provider_address, 2023, 6, 27, 6).transact(provider_dict)

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, requester_disease_code_list ).transact(provider_dict)

tx_hash = contract.functions.giveResearchPurpose(request_1_address, True, True, False, False, False).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, True, False, True, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, False, True).transact(requester_1_dict)
# tx_hash = contract.functions.GiveRequesterDate(request_1_address, 2023,6,29,2023,8,17).transact(requester_1_dict)

tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,provider_disease_code_list).transact(requester_1_dict)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert  result != 0


ABIFunctionNotFound: ("The function 'UploadProviderDiseasePurpose' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')

## Country


In [ ]:
import countrygroups as cg
import pycountry as pc
import os
import csv
import json

print(os.getcwd())
current_dir = os.getcwd()

country_fp = os.path.join(current_dir, "data/countries.csv")
country_line_fp = os.path.join(current_dir, "data/countries.jsonl")
country_json_fp = os.path.join(current_dir, "data/countries.json")
country_json_enrich_fp = os.path.join(current_dir, "data/countries_enrich.json")
group_index_fp = os.path.join(current_dir, "data/group_index.json")
country_code_rows = list(
    csv.DictReader(open(os.path.join(current_dir, country_fp), "r"))
)
# ['official_name_en']

print(cg.EUROPEAN_UNION)
country_code_dict = {row["ISO3166-1-Alpha-3"]: row for row in country_code_rows}
country_index_dict = {
    row["ISO3166-1-Alpha-3"]: i+1 for i, row in enumerate(country_code_rows)
}
group_name = [
    "ANNEX_ONE",
    "ANNEX_ONE_KAZ",
    "AOSIS",
    "AR5",
    "ARAB_GROUP",
    "BRICS",
    "EIG",
    "EUROPEAN_UNION",
    "G20",
    "G7",
    "G77",
    "GRADUATED_LDCS",
    "IMO",
    "LDC",
    "LLDC",
    "MONTREAL_PROTOCOL_HIGH_AMBIENT_TEMP",
    "NON_ANNEX_ONE",
    "OECD",
    "OPEC",
    "SIDS",
    "SIDS_NON_UN_OR_REGIONAL_COMMISSIONS_ASSOCIATES",
    "SSP",
    "UMBRELLA",
    "UN_REGIONAL_GROUPS",
    "UNFCCC",
    "UNSTATS_GEOGRAPHICAL_REGIONS",
    "MONTREAL_PROTOCOL_COUNTRIES",
]
gn_key = "groups"
group_index_dict = {gn: 2**i for i, gn in enumerate(group_name)}
with open(group_index_fp, "w") as f:
    f.write(json.dumps(group_index_dict, indent=4))

gn_simple_dict = {k: [] for k in country_code_dict.keys()}
for gn in group_name:
    countries_gn = cg.__dict__.get(gn)
    for country in countries_gn:
        gn_dict = country_code_dict.get(country)
        if gn_dict is None:
            print(gn)
            print(country)
            continue
        if gn_key not in gn_dict:
            gn_dict[gn_key] = list()
        gn_dict[gn_key].append(gn)

        if country not in gn_simple_dict:
            gn_simple_dict[country] = list()
        gn_simple_dict[country].append(gn)

with open(country_line_fp, "w") as f:
    text = ""
    for row in country_code_rows:
        text += json.dumps(row, ensure_ascii=False) + "\n"
    f.write(text)

with open(country_json_enrich_fp, "w") as f:
    need_keys = ["official_name_en", "ISO3166-1-Alpha-3", "groups"]
    enrich_dict = {}
    for k, v in country_code_dict.items():
        if "groups" not in v:
            v["groups"] = []
        enrich_dict[k] = {j: v[j] for j in need_keys}
        enrich_dict[k]["index"] = country_index_dict[k]

    f.write(json.dumps(enrich_dict, ensure_ascii=False, indent=4))

with open(country_json_fp, "w") as f:
    # country_group_dict = {k: v[gn_key] for k, v in country_code_dict.items()}
    f.write(json.dumps(gn_simple_dict, indent=4))

# print(country_code_rows[1])
# print(cg.__dict__.
#   keys())
# print(os.getcwd())

# print(pc.countries)


def index_for_requester(country_names: str = None, group_names: str = None):
    group_index = 0
    country_index = 0
    if group_names is not None:
        for gn in group_names:
            group_index += group_index_dict[gn]
    if country_names is not None:
        for cn in country_names:
            country_index += country_index_dict[cn]
            group_name_cn = country_code_dict[cn].get(gn_key)
            for gn in group_name_cn:
                group_index += group_index_dict[gn]
    return group_index, country_index


def index_for_provider(country_names: str = None, group_names: str = None):
    group_index = 0
    country_index = 0
    if group_names is not None:
        for gn in group_names:
            group_index += group_index_dict[gn]
    if country_names is not None:
        for cn in country_names:
            country_index += country_index_dict[cn]
    return group_index, country_index


# print(index_for_requester(group_names=['EUROPEAN_UNION']))
# print(index_for_requester(country_names=['CHN']))
# print(index_for_requester(country_names=['CHN'], group_names=['EUROPEAN_UNION']))
# print(index_for_requester(country_names=['CHN'], group_names=['EUROPEAN_UNION', 'G7']))
# print(index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7']))
# print(index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD']))

# print(index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20']))
# print(index_for_provider(country_names=['CHN'], group_names=['EUROPEAN_UNION']))
# print(index_for_provider(country_names=['CHN'], group_names=['EUROPEAN_UNION', 'G7']))
# print(index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION']))

d:\WorkSpace\Maastricht\LUCE\DecentralizedHealthcareBackend\jupyter
['AUT', 'BEL', 'BGR', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GRC', 'HRV', 'HUN', 'IRL', 'ITA', 'LTU', 'LUX', 'LVA', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'SWE']
ANNEX_ONE
EUU
ANNEX_ONE_KAZ
EUU
AR5
ANHH
G20
EUU
G7
EUU
UNFCCC
EUU
MONTREAL_PROTOCOL_COUNTRIES
EUU


In [ ]:
# GeographicSpecificRestriction is true, UseBySpecifiedCountries is True, Country meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, True, False, False, False, False,False,False,False).transact(provider_dict)
provider_group_index, provider_country_index = index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
tx_hash = contract.functions.UploadProviderCountry(provider_address, country_index, group_index).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,True).transact(requester_1_dict)
requester_group_index,requester_country_index = index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,country_index, group_index).transact(requester_1_dict)
OpenToNonProfitUseOnly = contract.functions.displayProviderDataRequirement(provider_address).call(provider_dict)
# tx_hash = Contract.functions.giveProfit(request_1_address,2).transact(requester_1_dict)
print("openNoProfit",OpenToNonProfitUseOnly)
requester_code = contract.functions.displayRequesterCountry( request_1_address).call()
provider_code = contract.functions.displayProviderCountry( provider_address).call()
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(f"requester_code is {requester_code}, provider_code is {provider_code} ")
print(result)
assert  result == 0

# GeographicSpecificRestriction is False ,  UseBySpecifiedCountries is True, Country not  meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, False, True, True, True,True,True,True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderCountry(provider_address, provider_group_index, provider_country_index).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,True).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,
                                                  requester_group_index,requester_country_index
                                                  ).transact(requester_1_dict)
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(result)
assert  result == 0

provider_group_index, provider_country_index = index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
requester_group_index,requester_country_index = index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
# GeographicSpecificRestriction is True ,  UseBySpecifiedCountries is False, Country not  meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, True, True, True, True, True,True,True,True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderCountry(provider_address, 
                                                   provider_group_index, provider_country_index
                                                   ).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,True).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,requester_group_index,requester_country_index
                                                  ).transact(requester_1_dict)
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(result)
assert  result == 0

provider_group_index, provider_country_index = index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
requester_group_index,requester_country_index = index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
# GeographicSpecificRestriction is True ,  UseBySpecifiedCountries is False, Country   meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, True, True, True, True,True,True,True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderCountry(provider_address, 
                                                   provider_group_index, provider_country_index 
                                                   ).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,False).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,
                                                  requester_group_index,requester_country_index
                                                  ).transact(requester_1_dict)
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(result)
assert  result == 0



: 

## Others

In [ ]:
startTime14 = datetime.now()
#Data Requesters Account
# request_1_address = accounts[0]
#Data Requester 2 (HMB and Profit Making)
tx_hash = contract.functions.giveHMBPurpose(request_1_address, True, True, True, False, True, True).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, False, True, True, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, True, False).transact(requester_1_dict)
endTimeDataRequester14 = datetime.now()
elapsedTimeDataRequester14 = endTimeDataRequester14 - startTime14
print("elapsedTimeDataRequester14", elapsedTimeDataRequester14)
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData14 = datetime.now()
#elapsedTimeDataRequesterAccessData14 = endTimeDataRequesterAccessData14 - startTime14
#print("elapsedTimeDataRequesterAccessData14", elapsedTimeDataRequesterAccessData14)
startTime15 = datetime.now()
#Data Requesters Account  
request_1_address = accounts[0]
#Data Requester 3 (Disease Specific and Non-Profit-Making)
tx_hash = contract.functions.giveHMBPurpose(request_1_address, False, False, False, True, False, False).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, False, True, False, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, False, True).transact(requester_1_dict)
endTimeDataRequester15 = datetime.now()
elapsedTimeDataRequester15 = endTimeDataRequester15 - startTime15
print("elapsedTimeDataRequester15", elapsedTimeDataRequester15)
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData15 = datetime.now()
elapsedTimeDataRequesterAccessData15 = endTimeDataRequesterAccessData15 - startTime15
print("elapsedTimeDataRequesterAccessData15", elapsedTimeDataRequesterAccessData15)
startTime1 = datetime.now()
#Data provider consent (General research & profit making)
#D-002
# test_address1 = accounts[1]
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
endTimeDataProvider1 = datetime.now()
elapsedTimeDataProvider1 = endTimeDataProvider1 - startTime1
print("elapsedTimeDataProvider1", elapsedTimeDataProvider1)
startTime1_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData1_13 = datetime.now()
elapsedTimeDataRequesterAccessData1_13 = endTimeDataRequesterAccessData1_13 - startTime1_13
print("elapsedTimeDataRequesterAccessData1_13", elapsedTimeDataRequesterAccessData1_13)
startTime1_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData1_14 = datetime.now()
elapsedTimeDataRequesterAccessData1_14 = endTimeDataRequesterAccessData1_14 - startTime1_14
print("elapsedTimeDataRequesterAccessData1_14", elapsedTimeDataRequesterAccessData1_14)
startTime1_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData1_15 = datetime.now()
elapsedTimeDataRequesterAccessData1_15 = endTimeDataRequesterAccessData1_15 - startTime1_15
print("elapsedTimeDataRequesterAccessData1_15", elapsedTimeDataRequesterAccessData1_15)
#startTime2 = datetime.now()
#Data provider consent (General research & non-profit making)
#H-001 
# test_address2 = accounts[2]

tx_hash = contract.functions.UploadDataPrimaryCategory(request_1_address, False, True, False, False, False).transact(requester_1_dict)
tx_hash = contract.functions.UploadDataRequirements(request_1_address, False, True, False, False, False, False, False, False).transact(requester_1_dict)
#endTimeDataProvider2 = datetime.now()
#elapsedTimeDataProvider2 = endTimeDataProvider2 - startTime2
#print("elapsedTimeDataProvider2", elapsedTimeDataProvider2)
#startTime2_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData2_13 = datetime.now()
#elapsedTimeDataRequesterAccessData2_13 = endTimeDataRequesterAccessData2_13 - startTime2_13
#print("elapsedTimeDataRequesterAccessData2_13", elapsedTimeDataRequesterAccessData2_13)
#startTime2_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData2_14 = datetime.now()
#elapsedTimeDataRequesterAccessData2_14 = endTimeDataRequesterAccessData2_14 - startTime2_14
#print("elapsedTimeDataRequesterAccessData2_14", elapsedTimeDataRequesterAccessData2_14)
#startTime2_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData2_15 = datetime.now()
#elapsedTimeDataRequesterAccessData2_15 = endTimeDataRequesterAccessData2_15 - startTime2_15
#print("elapsedTimeDataRequesterAccessData2_15", elapsedTimeDataRequesterAccessData2_15)#
#startTime3 = datetime.now()
#Data provider consent (General research & profit making)
#H-002 
test_address3 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, False, False, False).transact(provider_dict)
#endTimeDataProvider3 = datetime.now()
#elapsedTimeDataProvider3 = endTimeDataProvider3 - startTime3
#print("elapsedTimeDataProvider3", elapsedTimeDataProvider3)
#startTime3_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData3_13 = datetime.now()
#elapsedTimeDataRequesterAccessData3_13 = endTimeDataRequesterAccessData3_13 - startTime3_13
#print("elapsedTimeDataRequesterAccessData3_13", elapsedTimeDataRequesterAccessData3_13)
#startTime3_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData3_14 = datetime.now()
#elapsedTimeDataRequesterAccessData3_14 = endTimeDataRequesterAccessData3_14 - startTime3_14
#print("elapsedTimeDataRequesterAccessData3_14", elapsedTimeDataRequesterAccessData3_14)
#startTime3_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData3_15 = datetime.now()
#elapsedTimeDataRequesterAccessData3_15 = endTimeDataRequesterAccessData3_15 - startTime3_15
#print("elapsedTimeDataRequesterAccessData3_15", elapsedTimeDataRequesterAccessData3_15)
#startTime4 = datetime.now()
#Data provider consent (HMB research & non-profit making)
#D-003 
test_address4 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address4, False, False, True, False, True).transact(requester_1_dict)
tx_hash = contract.functions.UploadDataRequirements(test_address4, False, True, False, False, False, False, False, False).transact(requester_1_dict)

#endTimeDataProvider4 = datetime.now()
#elapsedTimeDataProvider4 = endTimeDataProvider4 - startTime4
#print("elapsedTimeDataProvider4", elapsedTimeDataProvider4)
#startTime4_13 = datetime.now()
contract.functions.AccessData(test_address4, request_1_address).call()
#endTimeDataRequesterAccessData4_13 = datetime.now()
#elapsedTimeDataRequesterAccessData4_13 = endTimeDataRequesterAccessData4_13 - startTime4_13
#print("elapsedTimeDataRequesterAccessData4_13", elapsedTimeDataRequesterAccessData4_13)
#startTime4_14 = datetime.now()
contract.functions.AccessData(test_address4, request_1_address).call()
#endTimeDataRequesterAccessData4_14 = datetime.now()
#elapsedTimeDataRequesterAccessData4_14 = endTimeDataRequesterAccessData4_14 - startTime4_14
#print("elapsedTimeDataRequesterAccessData4_14", elapsedTimeDataRequesterAccessData4_14)
#startTime4_15 = datetime.now()
contract.functions.AccessData(test_address4, request_1_address).call()
#endTimeDataRequesterAccessData4_15 = datetime.now()
#elapsedTimeDataRequesterAccessData4_15 = endTimeDataRequesterAccessData4_15 - startTime4_15
#print("elapsedTimeDataRequesterAccessData4_15", elapsedTimeDataRequesterAccessData4_15)
startTime5 = datetime.now()
#Data provider consent (HMB research & profit making)
#D-004 
test_address5 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address5, False, False, True, False, True).transact(requester_1_dict)
endTimeDataProvider5 = datetime.now()
elapsedTimeDataProvider5 = endTimeDataProvider5 - startTime5
print("elapsedTimeDataProvider5", elapsedTimeDataProvider5)
startTime5_13 = datetime.now()
contract.functions.AccessData(test_address5, request_1_address).call()
endTimeDataRequesterAccessData5_13 = datetime.now()
elapsedTimeDataRequesterAccessData5_13 = endTimeDataRequesterAccessData5_13 - startTime5_13
print("elapsedTimeDataRequesterAccessData5_13", elapsedTimeDataRequesterAccessData5_13)
startTime5_14 = datetime.now()
contract.functions.AccessData(test_address5, request_1_address).call()
endTimeDataRequesterAccessData5_14 = datetime.now()
elapsedTimeDataRequesterAccessData5_14 = endTimeDataRequesterAccessData5_14 - startTime5_14
print("elapsedTimeDataRequesterAccessData5_14", elapsedTimeDataRequesterAccessData5_14)
startTime5_15 = datetime.now()
contract.functions.AccessData(test_address5, request_1_address).call()
endTimeDataRequesterAccessData5_15 = datetime.now()
elapsedTimeDataRequesterAccessData5_15 = endTimeDataRequesterAccessData5_15 - startTime5_15
print("elapsedTimeDataRequesterAccessData5_15", elapsedTimeDataRequesterAccessData5_15)
#Data provider consent (HMB research & non-profit making)
#H-003 
test_address6 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address6, False, False, True, False, False).transact(requester_1_dict)
tx_hash = contract.functions.UploadDataRequirements(test_address6, False, True, False, False, False, False, False, False).transact(requester_1_dict)
contract.functions.AccessData(test_address6, request_1_address).call()
contract.functions.AccessData(test_address6, request_1_address).call()
contract.functions.AccessData(test_address6, request_1_address).call()
#Data provider consent (HMB research & profit making)
#H-004 
test_address7 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address7, False, False, True, False, False).transact(requester_1_dict)
contract.functions.AccessData(test_address7, request_1_address).call()
contract.functions.AccessData(test_address7, request_1_address).call()
contract.functions.AccessData(test_address7, request_1_address).call()
startTime8 = datetime.now()
#Data provider consent (Disease-specific research & non-profit making)
#D-005 

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, True).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, True, False, False, False, False, False, False).transact(provider_dict)
endTimeDataProvider8 = datetime.now()
elapsedTimeDataProvider8 = endTimeDataProvider8 - startTime8
print("elapsedTimeDataProvider8", elapsedTimeDataProvider8)
#Specify start and end dates

tx_hash = contract.functions.UploadProviderDate(provider_address, 2023,6,11,3).transact(provider_dict)
endTimeDataProvider8 = datetime.now()
elapsedTimeDataProvider8 = endTimeDataProvider8 - startTime8
print("elapsedTimeDataProvider8", elapsedTimeDataProvider8)
startTime8_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData8_13 = datetime.now()
elapsedTimeDataRequesterAccessData8_13 = endTimeDataRequesterAccessData8_13 - startTime8_13
print("elapsedTimeDataRequesterAccessData8_13", elapsedTimeDataRequesterAccessData8_13)
startTime8_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData8_14 = datetime.now()
elapsedTimeDataRequesterAccessData8_14 = endTimeDataRequesterAccessData8_14 - startTime8_14
print("elapsedTimeDataRequesterAccessData8_14", elapsedTimeDataRequesterAccessData8_14)
startTime8_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData8_15 = datetime.now()
elapsedTimeDataRequesterAccessData8_15 = endTimeDataRequesterAccessData8_15 - startTime8_15
print("elapsedTimeDataRequesterAccessData8_15", elapsedTimeDataRequesterAccessData8_15)
#Data provider consent (Disease-specific research & profit making)
# D-006 

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, True).transact(provider_dict)
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
#Data provider consent (Disease-specific research & non-profit making)
#H-005 
provider_address = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, False).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, True, False, False, False, False, False, False).transact(provider_dict)
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
#Data provider consent (Disease-specific research & profit making)
#H-006 
provider_address = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, False).transact(provider_dict)
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
endTime1 = datetime.now()
elapsedTime1 = endTime1 - startTime
print("elapsedTime1", elapsedTime1)
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
#print("Gas", Contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, True).estimateGas())
#print("Gas", Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).estimateGas())
#print("Gas", Contract.functions.giveResearchPurpose(request_1_address, True, True, False, False, False).estimateGas())
#print("Gas", Contract.functions.givePerson(request_1_address, True, False, True, False).estimateGas())
#print("Gas", Contract.functions.giveProfit(request_1_address, True, False).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, request_1_address).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, request_1_address).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, request_1_address).estimateGas())

: 